In [1]:
import json
import pandas as pd
from PIL import Image
from torchvision import transforms
import torch
from tqdm import tqdm
import re
import os
import shutil
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_path = "/content/drive/MyDrive/Deep Learning Project Group/Data"
print(os.listdir(data_path))

['radgraphchexpertresults', 'df_chexpert_plus_240401.csv', 'chexpert_reports_frontal_only.csv', 'chexpert_reports_frontal_pa_only.csv', 'chexpert_reports_frontal_pa_only.gsheet', 'Data Pre-Processing.ipynb']


In [ ]:
chexpert_plus_df = pd.read_csv(os.path.join(data_path, 'df_chexpert_plus_240401.csv'))
chexpert_plus_df.head()

,path_to_image,path_to_dcm,frontal_lateral,ap_pa,deid_patient_id,patient_report_date_order,report,section_narrative,section_clinical_history,section_history,...,section_accession_number,age,sex,race,ethnicity,interpreter_needed,insurance_type,recent_bmi,deceased,split
0,train/patient42142/study5/view1_frontal.jpg,train/patient42142/study5/view1_frontal.dcm,Frontal,AP,patient42142,5,"NARRATIVE:\nChest 1 View, 8-8-2005\n \nHISTORY...","\nChest 1 View, 8-8-2005\n \n",NaN,"61 years Female, ICU patient\n \n",...,\n 9959089\nThis report has been anonymized. A...,62.0,Female,White,Non-Hispanic/Non-Latino,No,Private Insurance,22.2,No,train
1,train/patient42142/study8/view1_frontal.jpg,train/patient42142/study8/view1_frontal.dcm,Frontal,AP,patient42142,8,"NARRATIVE:\nChest 1 View, 7-11-2000\n \nHISTOR...","\nChest 1 View, 7-11-2000\n \n",NaN,"61 years Female, tracheostomy.\n \n",...,\n64048857\nThis report has been anonymized. A...,62.0,Female,White,Non-Hispanic/Non-Latino,No,Private Insurance,22.2,No,train
2,train/patient42142/study2/view1_frontal.jpg,train/patient42142/study2/view1_frontal.dcm,Frontal,AP,patient42142,2,"NARRATIVE:\nChest 1 View, 11-17-2018\n \nHISTO...","\nChest 1 View, 11-17-2018\n \n",NaN,"61 years Female, critical care follow-up\n \n",...,\n#2452\nThis report has been anonymized. All ...,62.0,Female,White,Non-Hispanic/Non-Latino,No,Private Insurance,22.2,No,train
3,train/patient42142/study4/view1_frontal.jpg,train/patient42142/study4/view1_frontal.dcm,Frontal,AP,patient42142,4,NARRATIVE:\nAP PORTABLE UPRIGHT CHEST: May 01 ...,\nAP PORTABLE UPRIGHT CHEST: May 01 at 0518 ho...,ICU protocol. Follow up.\n \n,NaN,...,\n4164064\nThis report has been anonymized. Al...,62.0,Female,White,Non-Hispanic/Non-Latino,No,Private Insurance,22.2,No,train
4,train/patient42142/study3/view1_frontal.jpg,train/patient42142/study3/view1_frontal.dcm,Frontal,AP,patient42142,3,"NARRATIVE:\nEXAM: Chest 1 View, 10/4/2\n \nCLI...","\nEXAM: Chest 1 View, 10/4/2\n \n",61 years Female UPRIGHT PLEASE. ICU PROTOCOL ...,NaN,...,\n49286401\nThis report has been anonymized. A...,62.0,Female,White,Non-Hispanic/Non-Latino,No,Private Insurance,22.2,No,train


### Text Pre-processing & Feature Engineering

In [ ]:
chexpert_plus_df.iloc[0]['section_impression']

'  \n \n1.TRACHEOSTOMY TUBE REMAINS IN PLACE.  RIGHT PICC IS IN STABLE AND \nSTANDARD POSITION.  THE ENTERIC TUBE HAS BEEN REMOVED.\n \n2.NO EVIDENCE OF PNEUMOTHORAX.  SLIGHTLY IMPROVED AERATION OF THE \nBILATERAL LUNGS WITH NO EVIDENCE OF FOCAL AIR SPACE OPACITIES OR \nPLEURAL EFFUSIONS.\n \n '

In [ ]:
chexpert_plus_df.iloc[0]['report']

'NARRATIVE:\nChest 1 View, 8-8-2005\n \nHISTORY: 61 years Female, ICU patient\n \nCOMPARISON: 8/8/2005.\n \nIMPRESSION:  \n \n1.TRACHEOSTOMY TUBE REMAINS IN PLACE.  RIGHT PICC IS IN STABLE AND \nSTANDARD POSITION.  THE ENTERIC TUBE HAS BEEN REMOVED.\n \n2.NO EVIDENCE OF PNEUMOTHORAX.  SLIGHTLY IMPROVED AERATION OF THE \nBILATERAL LUNGS WITH NO EVIDENCE OF FOCAL AIR SPACE OPACITIES OR \nPLEURAL EFFUSIONS.\n \n SUMMARY: 2-ABNORMAL, PREVIOUSLY REPORTED  \nI have personally reviewed the images for this examination and agreed\nwith the report transcribed above.\n \nACCESSION NUMBER:\n 9959089\nThis report has been anonymized. All dates are offset from the actual dates by a fixed interval associated with the patient.'

In [ ]:
chexpert_plus_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223462 entries, 0 to 223461
Data columns (total 27 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   path_to_image               223462 non-null  object 
 1   path_to_dcm                 223462 non-null  object 
 2   frontal_lateral             223462 non-null  object 
 3   ap_pa                       191071 non-null  object 
 4   deid_patient_id             223462 non-null  object 
 5   patient_report_date_order   223462 non-null  int64  
 6   report                      223462 non-null  object 
 7   section_narrative           223434 non-null  object 
 8   section_clinical_history    138866 non-null  object 
 9   section_history             38192 non-null   object 
 10  section_comparison          213125 non-null  object 
 11  section_technique           10732 non-null   object 
 12  section_procedure_comments  25669 non-null   object 
 13  section_findin

In [ ]:
chexpert_plus_df.ap_pa.value_counts()

,count
ap_pa,
AP,161622
PA,29432
LL,16
RL,1


In [ ]:
chexpert_plus_df = chexpert_plus_df[chexpert_plus_df['frontal_lateral'] == 'Frontal']
chexpert_plus_df = chexpert_plus_df[chexpert_plus_df['ap_pa'] == 'PA']  # Drop AP and lateral

In [ ]:
chexpert_plus_df.shape

(29432, 27)

In [ ]:
# Select the key columns for retrieval
final_chexpert_plus_df = chexpert_plus_df[['path_to_image', 'report', 'section_impression']].copy()

# Clean the text columns (remove extra newlines, trim whitespace)
for col in ['report', 'section_impression']:
    final_chexpert_plus_df[col] = final_chexpert_plus_df[col].astype(str).str.replace('\n', ' ').str.strip().str.lower()

# Drop rows with empty reports, if any
final_chexpert_plus_df = final_chexpert_plus_df[final_chexpert_plus_df['report'].notnull()]

# Final dataset
final_chexpert_plus_df.head()

,path_to_image,report,section_impression
9,train/patient04528/study1/view1_frontal.jpg,narrative: radiographic examination of the che...,1. interval development of a band of increase...
16,train/patient11162/study5/view1_frontal.jpg,narrative: two views of the chest 21 september...,slight interval increase in left pleural effus...
19,train/patient11162/study4/view1_frontal.jpg,narrative: chest two views: 10th april 2014 cl...,stable postoperative appearance of the mediast...
21,train/patient11162/study6/view1_frontal.jpg,narrative: two view chest: 2/21/2002. clinical...,1. stable post-surgical thorax with median ste...
33,train/patient02244/study2/view1_frontal.jpg,narrative: 2 views of the chest: 2-26-2019 at ...,1. unchanged position of a right internal jugu...


In [ ]:
final_chexpert_plus_df.iloc[5].report

'narrative: chest 2 views: 10/11/2021   history: 65 years male, evaluate for infiltrate.   comparison: 21-10-11   impression:    1.  there is interval resolution of linear opacity at the left lung  base, favoring resolved discoid atelectasis.  there is no  consolidation or pleural effusion to suggest ongoing infectious  process.     2.  stable right central venous catheter with tip at the cavoatrial  junction.  cardiomediastinal silhouette and pulmonary vasculature are  within normal limits.   3.  old fracture or of the right ninth rib laterally.   summary:1-no significant abnormality   accession number: #8515800947 this report has been anonymized. all dates are offset from the actual dates by a fixed interval associated with the patient.'

In [ ]:
final_chexpert_plus_df.iloc[5].section_impression

'1.  there is interval resolution of linear opacity at the left lung  base, favoring resolved discoid atelectasis.  there is no  consolidation or pleural effusion to suggest ongoing infectious  process.     2.  stable right central venous catheter with tip at the cavoatrial  junction.  cardiomediastinal silhouette and pulmonary vasculature are  within normal limits.   3.  old fracture or of the right ninth rib laterally.'

In [ ]:
final_chexpert_plus_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29432 entries, 9 to 223458
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   path_to_image       29432 non-null  object
 1   report              29432 non-null  object
 2   section_impression  29432 non-null  object
dtypes: object(3)
memory usage: 919.8+ KB


In [ ]:
#final_chexpert_plus_df.to_csv('/Users/sohammandal/Class/ML2/Project/chexpert_reports_frontal_pa_only.csv', index=False)

In [ ]:
final_chexpert_plus_df.path_to_image.nunique()

29432

In [ ]:
final_chexpert_plus_df.shape

(29432, 3)

In [ ]:
final_chexpert_plus_df.iloc[0]['report']

'narrative: radiographic examination of the chest: 11-28-2005   clinical history: 61 years of age, female, r o infiltarate..aml  (acute myeloblastic leukemia)   comparison: 11/28/2005   procedure comments: two views of the chest.    findings:   unchanged position of the left upper extremity picc line. again seen  are surgical clips projecting over the right hemithorax. the  cardiomediastinal silhouette is stable in appearance. increased  stranding opacities are noted in the left retrocardiac region. subtle  stranding opacities in the right upper lung zone are unchanged..  there are no pleural or significant bony abnormalities. absence of  the right breast shadow compatible with prior mastectomy.   impression:   1.  interval development of a band of increased linear stranding  opacities in the left retrocardiac region. although this may  represent subsegmental atelectasis, an early or developing  consolidation could have similar appearance. recommend clinical  correlation.       accessi

In [ ]:
chexpert_pa_df = pd.read_csv(os.path.join(data_path, 'chexpert_reports_frontal_pa_only.csv'))
chexpert_pa_df.head()

,path_to_image,report,section_impression
0,train/patient04528/study1/view1_frontal.jpg,narrative: radiographic examination of the che...,1. interval development of a band of increase...
1,train/patient11162/study5/view1_frontal.jpg,narrative: two views of the chest 21 september...,slight interval increase in left pleural effus...
2,train/patient11162/study4/view1_frontal.jpg,narrative: chest two views: 10th april 2014 cl...,stable postoperative appearance of the mediast...
3,train/patient11162/study6/view1_frontal.jpg,narrative: two view chest: 2/21/2002. clinical...,1. stable post-surgical thorax with median ste...
4,train/patient02244/study2/view1_frontal.jpg,narrative: 2 views of the chest: 2-26-2019 at ...,1. unchanged position of a right internal jugu...


In [ ]:
def clean_impression(text):
    if pd.isnull(text):
        return ""

    text = text.lower()

    # Remove list numbers like "1.", "2)", "3 -", etc.
    text = re.sub(r'\b\d+[\.\)\-]\s*', '', text)

    # Remove weird special characters (but keep . , - / %)
    text = re.sub(r"[^a-z0-9\.\,\-/\%\s]", "", text)

    # Collapse multiple spaces and newlines
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

In [ ]:
chexpert_pa_df['cleaned_impression'] = chexpert_pa_df['section_impression'].apply(clean_impression)

In [ ]:
pd.set_option('display.max_colwidth', None)
chexpert_pa_df.head(20)

,path_to_image,report,section_impression,cleaned_impression
0,train/patient04528/study1/view1_frontal.jpg,"narrative: radiographic examination of the chest: 11-28-2005 clinical history: 61 years of age, female, r o infiltarate..aml (acute myeloblastic leukemia) comparison: 11/28/2005 procedure comments: two views of the chest. findings: unchanged position of the left upper extremity picc line. again seen are surgical clips projecting over the right hemithorax. the cardiomediastinal silhouette is stable in appearance. increased stranding opacities are noted in the left retrocardiac region. subtle stranding opacities in the right upper lung zone are unchanged.. there are no pleural or significant bony abnormalities. absence of the right breast shadow compatible with prior mastectomy. impression: 1. interval development of a band of increased linear stranding opacities in the left retrocardiac region. although this may represent subsegmental atelectasis, an early or developing consolidation could have similar appearance. recommend clinical correlation. accession number: 716611072 this report has been anonymized. all dates are offset from the actual dates by a fixed interval associated with the patient.","1. interval development of a band of increased linear stranding opacities in the left retrocardiac region. although this may represent subsegmental atelectasis, an early or developing consolidation could have similar appearance. recommend clinical correlation.","interval development of a band of increased linear stranding opacities in the left retrocardiac region. although this may represent subsegmental atelectasis, an early or developing consolidation could have similar appearance. recommend clinical correlation."
1,train/patient11162/study5/view1_frontal.jpg,"narrative: two views of the chest 21 september: clinical history: 61 year-old man status post heart transplant. comparison: 9/9/2021 and 21-09-09 findings: persistent small bilateral pleural effusions. the left effusion is slightly greater when compared to the prior exam. lungs are clear. stable cardiomediastinal silhouette . calcifications are again seen in the right suprahilar region, which may represent calcified lymph nodes or post surgical change. bones are stable with mild degenerative changes of the spine. sternotomy wires are unchanged. impression: slight interval increase in left pleural effusion. otherwise, no significant interval change. end of impression: summary:2-abnormal, previously reported accession number: 04979080 this report has been anonymized. all dates are offset from the actual dates by a fixed interval associated with the patient.","slight interval increase in left pleural effusion. otherwise, no significant interval change.","slight interval increase in left pleural effusion. otherwise, no significant interval change."
2,train/patient11162/study4/view1_frontal.jpg,"narrative: chest two views: 10th april 2014 clinical history: a 61-year-old male status post heart transplant. comparison: 4-10-14 impression: stable postoperative appearance of the mediastinum. bilateral tiny pleural effusions or pleural thickening persist. no pulmonary edema. the lungs are clear. end of impression: summary: 2-abnormal, previously reported accession number: 90839335779 this report has been anonymized. all dates are offset from the actual dates by a fixed interval associated with the patient.",stable postoperative appearance of the mediastinum. bilateral tiny pleural effusions or pleural thickening persist. no pulmonary edema. the lungs are clear.,stable postoperative appearance of the mediastinum. bilateral tiny pleural effusions or pleural thickening persist. no pulmonary edema. the lungs are clear.
3,train/patient11162/study6/view1_frontal.jpg,"narrative: two view chest: 2/21/2002. clinical data: heart transplant. comparison: 2/21/2002. impression: 1. stable post-surgical thorax with median sternotomy wires and surgical clips overlying the left hilum. dense calcific

In [ ]:
# Create the new DataFrame
image_text_df = chexpert_pa_df[['path_to_image', 'cleaned_impression']].copy()

# Save to a CSV file
#image_text_df.to_csv('/content/drive/MyDrive/Deep Learning Project Group/CheXRetriever/data/metadata.csv', index=False)

### Image Pre-processing

In [ ]:
data_path = "/content/drive/MyDrive/Deep Learning Project Group/CheXRetriever/data"
print(os.listdir(data_path))

['archive.zip', 'metadata.csv']


In [ ]:
zip_path = "/content/drive/MyDrive/Deep Learning Project Group/CheXRetriever/data/archive.zip"

In [ ]:
import zipfile

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/unzipped_final/")

print("Unzipped into /content/unzipped_final")

Unzipped into /content/unzipped_final


In [ ]:
# Load filtered metadata
df = pd.read_csv("content/drive/MyDrive/Deep Learning Project Group/CheXRetriever/data/metadata.csv")

# Paths
unzipped_root = "/content/unzipped_final"
final_images_root = "/content/chexpert_frontal_images"
os.makedirs(final_images_root, exist_ok=True)

missing_count = 0
copied_count = 0

for rel_path in df["path_to_image"]:
    src_path = os.path.join(unzipped_root, rel_path)  # Full path to the *exact* image
    dst_filename = rel_path.replace("/", "_")         # Flatten for final copy
    dst_path = os.path.join(final_images_root, dst_filename)

    # Only copy if the exact file exists
    if os.path.isfile(src_path):
        shutil.copy2(src_path, dst_path)
        copied_count += 1
    else:
        print(f"Missing file: {src_path}")
        missing_count += 1

print(f"Copied {copied_count} images from metadata.")
print(f"{missing_count} listed images not found in unzipped folder.")

Done copying frontal images. Missing files: 0


In [ ]:
# Sanity check to ensure images in chexpert_frontal_images match path_to_image in metadata

# Convert path_to_image to flattened filenames
df['flattened_filename'] = df['path_to_image'].apply(lambda x: x.replace("/", "_"))

# List of actual files in the image folder
image_dir = "/content/chexpert_frontal_images"
copied_filenames = set(os.listdir(image_dir))

# Sanity check: which expected files are missing?
expected_filenames = set(df['flattened_filename'])

missing = expected_filenames - copied_filenames
extra = copied_filenames - expected_filenames

print(f"Found {len(expected_filenames & copied_filenames)} matching files")
print(f"{len(missing)} expected images missing")
print(f"{len(extra)} unexpected images present")

Found 29432 matching files
0 expected images missing
0 unexpected images present


In [ ]:
# Define transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3)
])

# Directories
input_dir = '/content/chexpert_frontal_images'
output_dir = '/content/chexpert_images_transformed'
os.makedirs(output_dir, exist_ok=True)

# Transform and save
print("Applying transformations...")
for filename in tqdm(os.listdir(input_dir)):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        input_path = os.path.join(input_dir, filename)
        output_filename = os.path.splitext(filename)[0] + '.png'  # force PNG output
        output_path = os.path.join(output_dir, output_filename)
        try:
            image = Image.open(input_path).convert("L")  # Assure grayscale
            image = transform(image)
            image.save(output_path, format='PNG')
        except Exception as e:
            print(f"💀 Error on {filename}: {e}")

print("Lossless PNGs saved in chexpert_transformed/")

Applying transformations...


100%|██████████| 29432/29432 [10:54<00:00, 44.94it/s]

Lossless PNGs saved in chexpert_transformed/


### Train-Test-Val Split

In [ ]:
print(os.listdir(data_path))

['archive.zip', 'metadata.csv', 'val', 'test', 'train']


In [ ]:
# Load metadata
df = pd.read_csv('/content/drive/MyDrive/Deep Learning Project Group/CheXRetriever/data/metadata.csv')

# First split: train (70%) and temp (30%)
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, shuffle=True)

# Second split: temp into val (15%) and test (15%)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, shuffle=True)

# Save the splits
train_df.to_csv('/content/drive/MyDrive/Deep Learning Project Group/CheXRetriever/data/train/train_metadata.csv', index=False)
val_df.to_csv('/content/drive/MyDrive/Deep Learning Project Group/CheXRetriever/data/val/val_metadata.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/Deep Learning Project Group/CheXRetriever/data/test/test_metadata.csv', index=False)

print("Metadata split complete:")
print(f"Train: {len(train_df)} samples")
print(f"Val: {len(val_df)} samples")
print(f"Test: {len(test_df)} samples")

🧾 Metadata split complete:
Train: 20602 samples
Val: 4415 samples
Test: 4415 samples


In [ ]:
# Directories
split_dir = "/content/drive/MyDrive/Deep Learning Project Group/CheXRetriever/data/"
transformed_dir = "/content/chexpert_images_transformed"

# Metadata files
metadata_files = {
    "train": "/content/drive/MyDrive/Deep Learning Project Group/CheXRetriever/data/train/train_metadata.csv",
    "val": "/content/drive/MyDrive/Deep Learning Project Group/CheXRetriever/data/val/val_metadata.csv",
    "test": "/content/drive/MyDrive/Deep Learning Project Group/CheXRetriever/data/test/test_metadata.csv"
}

# Flattening function for path match
def flatten_path(path):
    return path.replace("/", "_").replace("\\", "_").replace(".jpg", ".png")

# Copying function
def process_split(split_name, metadata_path):
    print(f"\nProcessing split: {split_name}")

    output_image_dir = os.path.join(split_dir, split_name, "images")
    os.makedirs(output_image_dir, exist_ok=True)

    df = pd.read_csv(metadata_path)

    copied = 0
    missing = 0

    for rel_path in tqdm(df["path_to_image"]):
        flat_filename = flatten_path(rel_path)
        src_path = os.path.join(transformed_dir, flat_filename)
        dst_path = os.path.join(output_image_dir, flat_filename)

        if os.path.isfile(src_path):
            shutil.copy2(src_path, dst_path)
            copied += 1
        else:
            print(f"Missing file: {src_path}")
            missing += 1

    print(f"{split_name}: Copied {copied} images.")
    print(f"{split_name}: {missing} images were missing.")

# Run for all splits
for split, metadata_path in metadata_files.items():
    process_split(split, metadata_path)


Processing split: train


100%|██████████| 20602/20602 [05:41<00:00, 60.34it/s]


train: Copied 20602 images.
train: 0 images were missing.

Processing split: val


100%|██████████| 4415/4415 [01:09<00:00, 63.34it/s]


val: Copied 4415 images.
val: 0 images were missing.

Processing split: test


100%|██████████| 4415/4415 [01:09<00:00, 63.50it/s]

test: Copied 4415 images.
test: 0 images were missing.


In [ ]:
# Sanity check to ensure train images saved
train_dir = "/content/drive/MyDrive/Deep Learning Project Group/CheXRetriever/data/train/images"
print("Files in train/images:", os.listdir(train_dir)[:5])
print("Total files in train/images:", len(os.listdir(train_dir)))

Files in train/images: ['train_patient22959_study6_view1_frontal.png', 'train_patient20931_study1_view1_frontal.png', 'train_patient09530_study16_view1_frontal.png', 'train_patient03122_study42_view1_frontal.png', 'train_patient29439_study1_view1_frontal.png']
Total files in train/images: 20602


In [ ]:
# Sanity check to ensure test images saved
test_dir = "/content/drive/MyDrive/Deep Learning Project Group/CheXRetriever/data/test/images"
print("Files in test/images:", os.listdir(test_dir)[:5])
print("Total files in test/images:", len(os.listdir(test_dir)))

Files in test/images: ['train_patient31283_study2_view1_frontal.png', 'train_patient33150_study1_view1_frontal.png', 'train_patient27872_study1_view1_frontal.png', 'train_patient27903_study2_view2_frontal.png', 'train_patient20560_study6_view1_frontal.png']
Total files in test/images: 4415


In [ ]:
# Sanity check to ensure val images saved
val_dir = "/content/drive/MyDrive/Deep Learning Project Group/CheXRetriever/data/val/images"
print("Files in val/images:", os.listdir(val_dir)[:5])
print("Total files in val/images:", len(os.listdir(val_dir)))

Files in val/images: ['train_patient14534_study7_view1_frontal.png', 'train_patient29739_study1_view1_frontal.png', 'train_patient10731_study5_view1_frontal.png', 'train_patient34258_study3_view1_frontal.png', 'train_patient21682_study7_view1_frontal.png']
Total files in val/images: 4415


In [ ]:
# Sanity check to ensure the right images are being mapped for train, test and val

# Define paths
split_dir = "/content/drive/MyDrive/Deep Learning Project Group/CheXRetriever/data/"
splits = ["train", "val", "test"]

# Flatten function used in your actual pipeline
def flatten_path(path):
    return path.replace("/", "_").replace("\\", "_").replace(".jpg", ".png")

# Checker
for split in splits:
    print(f"\nChecking {split} split")

    meta_path = os.path.join(split_dir, split, f"{split}_metadata.csv")
    img_dir = os.path.join(split_dir, split, "images")

    df = pd.read_csv(meta_path)
    expected_images = set(df["path_to_image"].apply(flatten_path))
    actual_images = set(os.listdir(img_dir))

    # Check for missing images
    missing_images = expected_images - actual_images
    extra_images = actual_images - expected_images

    print(f"Metadata entries: {len(expected_images)}")
    print(f"Actual images found: {len(actual_images)}")
    print(f"Missing images: {len(missing_images)}")
    print(f"Extra images: {len(extra_images)}")

    if missing_images:
        print("Some missing images:")
        print(list(missing_images)[:5])

    if extra_images:
        print("Some extra images in folder not listed in metadata:")
        print(list(extra_images)[:5])

    if not missing_images and not extra_images:
        print("All good.")


Checking train split
Metadata entries: 20602
Actual images found: 20602
Missing images: 0
Extra images: 0
All good.

Checking val split
Metadata entries: 4415
Actual images found: 4415
Missing images: 0
Extra images: 0
All good.

Checking test split
Metadata entries: 4415
Actual images found: 4415
Missing images: 0
Extra images: 0
All good.
